In [1]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import YamlOutputParser

In [2]:
llm = Ollama(model="gemma:7b")

In [3]:
from langchain_core.prompts import ChatPromptTemplate
import re

def extract_cypher_scripts(text):
    cypher_list = list()
    for line in text.split("\n"):
        if 'CREATE' in line or 'MATCH' in line or 'CONNECT' in line:
            cypher_list.append(line)
            
    return cypher_list

## CoT Knowledge Grapg Creation

In [4]:
import pandas as pd
import numpy as np
cypher_list = list()
contexts = pd.read_csv('/Users/mertakcay/Projects/KnowledgeGraphBuilder/context.csv')
context_outputs = list()
for context in contexts['context']:
    prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a ontologist. You build knowledge graph from plain text. \n 
     Task 1: Extract Entities from plain text as possible as atomic. \n
     Task 2: Define Cypher Node Type to each Entity.\n
     Task 3: Add features of each Entity from plain text. \n
     Task 4: Extract relationship between Entities from plain text as possible as atomic. \n\n
     Task 5: Convert all outputs to Neo4j Cypher Creation Format"""),
    ("system", "Let's think step by step."),
    ("user", "{input}")
    ])
    chain = prompt | llm 
    output = chain.invoke({"input": context})
    print(output)
    print('--------------------------------')
    cypher_scripts = extract_cypher_scripts(output.replace('`','"'))
    context_outputs.append(cypher_scripts)
    

**Task 1: Extract Entities from Plain Text**

**Entities:**

- Beyoncé Giselle Knowles-Carter
- Houston, Texas
- Destiny's Child
- Mathew Knowles
- Dangerously in Love
- Crazy in Love
- Baby Boy

**Task 2: Define Cypher Node Type to Each Entity**

**Node Type:**

- **Person:** Beyoncé Giselle Knowles-Carter, Mathew Knowles
- **Location:** Houston, Texas
- **Group:** Destiny's Child
- **Album:** Dangerously in Love

**Task 3: Add Features of Each Entity from Plain Text**

**Features:**

- **Person:** Born in 1981, singer, songwriter, record producer, actress
- **Location:** Born and raised in Houston, Texas
- **Group:** Led singer, one of the world's best-selling girl groups of all time
- **Album:** Released in 2003, earned five Grammy Awards, featured number-one singles

**Task 4: Extract Relationships between Entities from Plain Text**

**Relationships:**

- Born and raised in
- Performed as
- Managed by
- Became
- Established

**Task 5: Convert All Outputs to Neo4j Cypher Creation Fo

In [11]:
import pickle 

def save_list_to_file(my_list, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(my_list, file)
    print(f'List saved to {file_path}')
    
def read_list_from_file(file_path):
    with open(file_path, 'rb') as file:
        my_list = pickle.load(file)
    print(f'List read from {file_path}')
    return my_list


In [12]:

save_path = './Automated_KG_list.pkl'
save_list_to_file(context_outputs, save_path)
context_read


List saved to ./Automated_KG_list.pkl
List read from ./Automated_KG_list.pkl


[["CREATE (b:Person {name: 'Beyoncé Giselle Knowles-Carter', born: 1981})",
  "CREATE (h:Location {name: 'Houston, Texas'})",
  "CREATE (g:Group {name: 'Destiny''s Child'})",
  "CREATE (m:Manager {name: 'Mathew Knowles'})",
  "CREATE (a:Album {name: 'Dangerously in Love'})",
  'CREATE (b)-[:BORN_AND_RAISED_IN]->(h)',
  'CREATE (b)-[:PERFORMED_AS]->(g)',
  'CREATE (b)-[:MANAGED_BY]->(m)',
  'CREATE (g)-[:BECAME]->(a)',
  'CREATE (a)-[:ESTABLISHED]->(b)'],
 ['CREATE (b:Artist {name: "Beyoncé"})',
  'CREATE (dc:Group {name: "Destiny\'s Child"})',
  'CREATE (d:Movie {name: "Dreamgirls"})',
  'CREATE (tp:Movie {name: "The Pink Panther"})',
  'CREATE (o:Movie {name: "Obsessed"})',
  'CREATE (cr:Movie {name: "Cadillac Records"})',
  'CREATE (sl:Song {name: "Single Ladies (Put a Ring on It)")',
  'CREATE Relationship (b) -[: RELEASED] - (bd)',
  'CREATE Relationship (b) -[: ACTED_IN] - (d)',
  'CREATE Relationship (b) -[: INFLUENCED] - (cr)'],
 ['CREATE (b:Artist {name: "Beyoncé"})',
  'CREATE

In [13]:
context_read[0]

["CREATE (b:Person {name: 'Beyoncé Giselle Knowles-Carter', born: 1981})",
 "CREATE (h:Location {name: 'Houston, Texas'})",
 "CREATE (g:Group {name: 'Destiny''s Child'})",
 "CREATE (m:Manager {name: 'Mathew Knowles'})",
 "CREATE (a:Album {name: 'Dangerously in Love'})",
 'CREATE (b)-[:BORN_AND_RAISED_IN]->(h)',
 'CREATE (b)-[:PERFORMED_AS]->(g)',
 'CREATE (b)-[:MANAGED_BY]->(m)',
 'CREATE (g)-[:BECAME]->(a)',
 'CREATE (a)-[:ESTABLISHED]->(b)']